### In this notebook we obtain the normal form of the benchmarking circuits

In [135]:
import numpy as np
import pytket as tk
from pytket import passes as tkp
from pytket import circuit as tkc
from pytket.circuit.display import render_circuit_jupyter as print_circ
from pytket.qasm import circuit_from_qasm_str, circuit_to_qasm,circuit_from_qasm
import zipfile
import os
import shutil

The documentation.ipynb notebook has through descriptions of the functions and their purpose. This notebook goes through the procedure for obtaining the normal form, <br>
which we apply to all the circuits in the benchmarking folder, "nam-circuits" in the original Quantinuum challenge repository

In [136]:
def adding_registers(reading_file, modified_file):
    '''
    Counts the number of H gates h_count, adds a flag at
    each hadamard, and creates a quantum register and classical
    register of size h_count.
    '''
    h_count = 0
    h_qbit_list = []
    with open(reading_file, 'r') as file, open('temp.qasm', 'w') as new_file:
        for line in file:
            if line.strip().startswith('h q['):
                h_count += 1
                start_idx = line.find('[')
                end_idx = line.find(']')
                h_qbit_list.append(int(line[start_idx+1:end_idx]))
                new_file.write('//spotted\n')
            new_file.write(line)
    file.close()
    new_file.close()

    with open('temp.qasm', 'r') as file, open(modified_file, 'w') as new_file:
        for line in file:
            if line.strip().startswith('qreg'):
                new_file.write(line)
                new_file.write(f'qreg a[{h_count}];\n')
                new_file.write(f'creg ma[{h_count}];\n')
            else:
                new_file.write(line)


    return h_qbit_list

def hadamard_string(qbit_number, ancilla_number):
    '''
    Creates the two qubit blocks that
    substitute the hadamard gates
    '''

    return          f'h a[{ancilla_number}];\n' \
                    +   f's q[{qbit_number}];\n' \
                    +   f's a[{ancilla_number}];\n' \
                    +   f'cx q[{qbit_number}], a[{ancilla_number}];\n' \
                    +   f'sdg a[{ancilla_number}];\n' \
                    +   f'cx a[{ancilla_number}], q[{qbit_number}];\n' \
                    +   f'cx q[{qbit_number}], a[{ancilla_number}];\n' \
                    +   f'h a[{ancilla_number}];\n' \
                    +   f'measure a[{ancilla_number}] -> ma[{ancilla_number}];\n' \
                    +   f'if (ma[{ancilla_number}] == 1) x q[{qbit_number}];\n'

def add_h_blocks(old_file, modified_file, h_qbit_list):
    '''
    Substitutes the hadamard gates
    by the two qubit blocks defined in the paper
    '''
    flag = False
    ancilla_idx = 0

    with open(old_file, 'r') as file, open(modified_file, 'w') as new_file:
        for line in file:
            if flag:
                flag = False
                continue
            else:
                pass

            if line.strip().startswith('//spotted'):
                flag = True
                new_file.write(line)
                new_file.write(hadamard_string(h_qbit_list[ancilla_idx], len(h_qbit_list) - 1 - ancilla_idx))
                ancilla_idx += 1
            else:
                new_file.write(line)



First we have to apply the notebook correcting_qasm_order over the circuit files to obtain them in the Clifford basis and <br>
in the proper order. From that, we obtain qasm_files.zip, which we have to unzip. 

In [137]:
# Extract the files of qasm_files.zip (quantum circuits in Clifford + T gateset) to a folder
extract_to = 'qasm_files'
zip_path = 'circuits/qasm_files.zip'
os.makedirs(extract_to)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract all the contents into the directory
    zip_ref.extractall(extract_to)
    print(f"Files have been extracted to {extract_to}")

Files have been extracted to qasm_files


Now we gadgetize all the circuits in the folder, and also save the positions of the hadamards in each circuit as a .txt file

In [138]:
#Function for gadgetizing all the qasm files

def gadgetize_qasm_files(source_dir, target_dir, log_dir):
    os.makedirs(target_dir, exist_ok=True)
    os.makedirs(log_dir, exist_ok=True)  # Ensure the log directory exists
    for filename in os.listdir(source_dir):
        if filename.endswith('.qasm'):
            reading_file = os.path.join(source_dir, filename)
            temp_file = os.path.join(target_dir, 'temp_' + filename)
            modified_file = os.path.join(target_dir, filename)
            h_qubit_list = adding_registers(reading_file, temp_file)
            add_h_blocks(temp_file, modified_file, h_qubit_list)
            os.remove(temp_file)

            # Create a log file for each qasm file with the same base name
            log_filename = os.path.join(log_dir, filename.replace('.qasm', '.txt'))
            with open(log_filename, 'w') as log_file:
                log_file.write(f"{h_qubit_list}\n")

source_dir = 'qasm_files/'
target_dir = 'qasm_gadgetized_files/'
log_dir = 'qasm_h_qubit_list_log/'  # Directory where individual log files will be saved
os.makedirs('U_s', exist_ok=True)
gadgetize_qasm_files(source_dir, target_dir, log_dir)

Now we check if the gadgetization worked, testing on one of the circuits in the folder

In [139]:
qc = circuit_from_qasm('qasm_files/barenco_tof_3_modified.qasm')
print_circ(qc)

qc_gadgetized = circuit_from_qasm('qasm_gadgetized_files/barenco_tof_3_modified.qasm')
print_circ(qc_gadgetized)


In [140]:
def return_Uf(filename, filename_2,folder_U, flag):
    with open(filename, 'r') as f:
        lines = f.readlines()
    f.close()
    counter = 0

    while True:
      for i in range(len(lines)-1, -1, -1):
          if flag in lines[i]:
              lines[i] = "//flag\n"
              break

      if i == 0:
          break

      else:
          del lines[i+8:i+11]
          with open(folder_U+f"/U_{counter}.qasm", 'w') as f:
              f.writelines(lines[:4])
              if counter:
                f.writelines([f'qreg a[{counter}];\n'])
              else:
                f.write('\n')
              f.writelines(lines[i+8:])
          counter += 1
          f.close()

    with open(filename_2, 'w') as f:
        f.writelines(lines)
    f.close()

def gen_u_dag_qasms(h_qbit_list,folder_U):
    h_num = len(h_qbit_list)
    for i in range(h_num):
        u = circuit_from_qasm(folder_U+f"/U_{i}.qasm")
        u_dag = u.dagger()
        circuit_to_qasm(u_dag,folder_U+f'/U_{i}_dag.qasm')

In [141]:
# IMPORTANT: Choose a gadgetized circuit! these are the only ones that are properly flagged
filename = 'qasm_gadgetized_files/tof_3_modified.qasm'
os.makedirs('qasm_diagonalizable_files', exist_ok=True)
os.makedirs('U_s/tof_3_modified', exist_ok=True)
residual_file = 'qasm_diagonalizable_files/tof_3_modified.qasm'
flag = '//spotted'
folder_U = 'U_s/tof_3_modified'

return_Uf(filename, residual_file, folder_U,flag)



In [142]:
def commute_all(residual_qasm, new_qasm, h_qbit_list,folder_U):
    with open(residual_qasm, 'r') as f:
        old_lines = f.readlines()
    f.close()

    with open(new_qasm, 'w') as new_f:

        new_f.writelines(old_lines)
        new_f.writelines('//so it begins\n')

        h_num = len(h_qbit_list)

        for i in range(h_num):
            idx = h_num - 1 - i

            with open(folder_U+f'/U_{idx}.qasm', 'r') as f1:
                lines1 = f1.readlines()
            f1.close()

            u = lines1[7:]

            with open(folder_U+f'/U_{idx}_dag.qasm', 'r') as f2:
                lines2 = f2.readlines()
            f2.close()
            u_dag = lines2[7:]


            
            h_string = f'h a[{idx}];\n'
            meas_string = f'measure a[{idx}] -> ma[{idx}];\n'
            control = f'if (ma[{idx}] == 1) '
            x_line = f'x q[{h_qbit_list[i]}];\n'

            new_f.write(h_string)
            new_f.write(meas_string)
            for _ in u_dag:
                if len(_) > 2 and _[:3] != '//f':
                    new_f.write(control + _)
            new_f.write(h_string)
            new_f.write(control + x_line)
            for _ in u:
                if len(_)> 2 and _[:3] != '//f':
                    new_f.write(control + _)

    new_f.close()

In [143]:
import ast
h_qubit_list = []
with open('qasm_h_qubit_list_log/tof_3_modified.txt', 'r') as file:
    h_qubit_list = ast.literal_eval(file.read().strip())
final_file = 'output.qasm'


gen_u_dag_qasms(h_qubit_list,folder_U)
commute_all(residual_file, final_file, h_qubit_list,folder_U)

In [158]:
c = tk.Circuit(5,5)
c.append(circuit_from_qasm(f'tof_3_modified.qasm'))
c.Measure(0,0)
c.Measure(1,1)
c.Measure(2,2)
c.Measure(3,3)
c.Measure(4,4)
print_circ(c)

cn = tk.Circuit(5,5)
cn.append(circuit_from_qasm('output.qasm'))
cn.Measure(0,0)
cn.Measure(1,1)
cn.Measure(2,2)
cn.Measure(3,3)
cn.Measure(4,4)
print_circ(cn)

## Obtaining definitive normal form circuit

In [153]:
from pytket.passes import ComposePhasePolyBoxes
from pytket.passes import DecomposeBoxes

def load_from_qasm(file_path):
    qc_from_qasm = circuit_from_qasm(file_path, maxwidth=1000)
    return qc_from_qasm

def diagonalize_cnot(circuit):
  phase_poly = ComposePhasePolyBoxes()
  phase_poly.apply(circuit)
  #print_circ(circuit)
  decompose = DecomposeBoxes()
  decompose.apply(circuit)
  #print_circ(circuit)
  return circuit

def extract_qasm_content_until_marker(file_path, marker, output_file):
    """
    Extracts lines from a QASM file up to a specified marker and writes to a new file.

    Parameters:
        file_path (str): Path to the input QASM file.
        marker (str): Marker to stop at (line containing this marker will not be included).
        output_file (str): Path to the output QASM file where the extracted content will be saved.
    """
    try:
        with open(file_path, 'r') as infile:
            content = []
            for line in infile:
                if marker in line:
                    break
                if line.find('h a'):
                  content.append(line.rstrip())  # Use rstrip to remove trailing newlines

        with open(output_file, 'w') as outfile:
            outfile.write('\n'.join(content) + '\n')  # Write with a newline between lines

        print(f"Content extracted and written to {output_file}")
    except FileNotFoundError:
        print(f"Error: The file {file_path} does not exist.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

def add_back_H(qasm_file, output_file):
  with open(qasm_file, 'r') as infile, open(output_file, 'w') as new_file:
    for line in infile:
      new_file.write(line)
      if not line.find('qreg a'):
        start_idx = line.find('[')
        end_idx = line.find(']')
        number_a = int(line[start_idx+1:end_idx])
        for a in range(number_a):
          new_file.write(f'h a[{a}]; \n')
  infile.close()
  new_file.close()

def reconstruct_full_circuit(input_file, firstpart_file, output_file, marker):
  flag = False
  with open(firstpart_file, 'r') as infile, open(output_file, 'w') as new_file:
    for line in infile:
      new_file.write(line)
  infile.close()
  new_file.close()
  with open(input_file, 'r') as infile, open(output_file, 'a') as new_file:
    for line in infile:
      if flag:
        new_file.write(line)
      if not line.find(marker):
        flag = True
  infile.close()
  new_file.close()



first_part_file = 'qasm_files/tof_3_modified.qasm'
c = load_from_qasm(first_part_file)
c = diagonalize_cnot(c)
circuit_to_qasm(c, 'first_part_diag.qasm')
add_back_H('first_part_diag.qasm', 'first_part_diag_H.qasm')

output_file = "final_circuit.qasm"
marker = "//so it begins"
firstpart_file = 'first_part_diag_H.qasm'
reconstruct_full_circuit(first_part_file, firstpart_file, output_file, marker)

In [156]:
cn = tk.Circuit(5,5)
cn.append(circuit_from_qasm('final_circuit.qasm'))
cn.Measure(0,0)
cn.Measure(1,1)
cn.Measure(2,2)
cn.Measure(3,3)
cn.Measure(4,4)
print_circ(cn)


In [146]:
# We could not run simulations on local, so we run them on Nexus